# Additional Modelling Challenges

## Warehouse Location Problem

A company is considering opening warehouses at some candidate locations in order to supply its existing stores.

1. The potential cities for these warehouses are New York, Los Angeles, Chicago, and Atlanta.
2. Each possible warehouse has the same maintenance cost, and a capacity designating the maximum number of stores that it can supply.
3. The supply cost to a store is different for each warehouse.
4. Each store must be supplied by exactly one open warehouse.

The company needs to decide which warehouses to open based on several constraints:

1. If the New York warehouse is opened, then the Los Angeles warehouse must also be opened.
2. No more than three warehouses can be operational.
3. Either the Atlanta or the Los Angeles warehouse must be opened.

The objective is to determine the optimal set of warehouses to open, and which of these warehouses should supply the various stores so that supply costs are minimized.
Supply costs for each warehouse-store combination should be given.

In [1]:
import cpmpy as cp
import random

random.seed(0)

# Parameters
# 1. The potential cities for these warehouses are New York, Los Angeles, Chicago, and Atlanta.
NYC, LA, CHI, ATL = 0, 1, 2, 3
max_n_warehouses = len([NYC, LA, CHI, ATL])
# 2. Each possible warehouse has the same maintenance cost, and a capacity designating the maximum number of stores that it can supply.
warehouse_maintenance_cost = 1
warehouse_capacity = random.choices(range(5, 15), k=max_n_warehouses)
print("Warehouse capacities:", warehouse_capacity)
# 3. The supply cost to a store is different for each warehouse.
n_stores = 25
store_supply_cost_by_warehouse = [random.choices(range(1, max_n_warehouses+1), k=max_n_warehouses) for _ in range(n_stores)]
print("Store supply costs by warehouse:", store_supply_cost_by_warehouse)

model = cp.Model()

# Decision variables
# 4. Each store must be supplied by exactly one open warehouse.
warehouse_open = cp.boolvar(shape=max_n_warehouses)
store_supplier = cp.intvar(0, max_n_warehouses-1, shape=n_stores)

# Constraints
for wh in range(max_n_warehouses):
    # 2. Each possible warehouse has the same maintenance cost, and a capacity designating the maximum number of stores that it can supply.
    model += cp.sum([store_supplier[st] == wh for st in range(n_stores)]) <= warehouse_capacity[wh]
    # 4. Each store must be supplied by exactly one open warehouse.
    model += [(store_supplier[st] == wh).implies(warehouse_open[wh]) for st in range(n_stores)]

# 1. If the New York warehouse is opened, then the Los Angeles warehouse must also be opened.
model += warehouse_open[NYC].implies(warehouse_open[LA])

# 2. No more than three warehouses can be operational.
model += warehouse_open.sum() <= 3

# 3. Either the Atlanta or the Los Angeles warehouse must be opened.
model += warehouse_open[ATL] | warehouse_open[LA]

# The objective is to determine the optimal set of warehouses to open, and which of these warehouses should supply the various stores so that supply costs are minimized.
cost = [cp.Element(store_supply_cost_by_warehouse[i], store_supplier[i]) for i in range(n_stores)]
total_cost = cp.sum(cost)
model.minimize(total_cost)

if model.solve():
    print("Solution found!")
    print("Open warehouses:", [key for key, value in {"NYC": NYC, "LA": LA, "CHI": CHI, "ATL": ATL}.items() if warehouse_open.value()[value]])
    print("Store supplied by warehouse:", store_supplier.value())
    print("Cost:", [c.value() for c in cost], "Total:", total_cost.value())
else:
    print("No solution found.")

Warehouse capacities: [13, 12, 9, 7]
Store supply costs by warehouse: [[3, 2, 4, 2], [2, 3, 4, 3], [2, 4, 3, 2], [4, 4, 4, 4], [2, 3, 4, 3], [2, 1, 2, 3], [4, 4, 2, 4], [2, 4, 3, 1], [3, 2, 4, 3], [1, 2, 4, 1], [2, 4, 1, 3], [1, 4, 4, 2], [1, 2, 3, 4], [1, 3, 3, 3], [4, 3, 4, 3], [3, 2, 3, 2], [3, 2, 1, 1], [3, 3, 2, 1], [4, 4, 4, 4], [4, 4, 3, 2], [3, 2, 4, 4], [4, 3, 4, 3], [2, 3, 4, 4], [4, 1, 3, 2], [3, 4, 1, 3]]
Solution found!
Open warehouses: ['NYC', 'LA', 'CHI']
Store supplied by warehouse: [1 0 0 1 0 1 2 0 1 0 2 0 0 0 1 1 2 2 0 2 1 1 0 1 2]
Cost: [2, 2, 2, 4, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 3, 2, 1, 2, 4, 3, 2, 3, 2, 1, 1] Total: 48
